In [8]:
from sentence_transformers import SentenceTransformer, util
import json
# Open json
with open("tmp.json") as f:
    chunks = json.load(f)

print(len(chunks))
chunks[0]

130


'INTERVIEWER:\ncall. Yeah wait the budgeting thing.'

In [10]:

query = "Do people specifically try to hire veterans?"
# docs = ["Around 9 Million people live in London. The grass there is very pretty and there is great food", "London is known for its financial district", "I am a big taco fan. My favorite cities for tacos are"]
docs = chunks
#Load the model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

#Encode query and documents
query_emb = model.encode(query)
doc_emb = model.encode(docs)

#Compute dot score between query and all document embeddings
scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()

#Combine docs & scores
doc_score_pairs = list(zip(docs, scores))

#Sort by decreasing score
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

#Output passages & scores
for doc, score in doc_score_pairs:
    print(score, doc)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
0.4060405194759369 INTERVIEWEE:
Yeah. Yeah, so we do, um, the first, like the biggest tool that I use, because honestly it's the most cost effective and I work at startup, like I've been at mid-stage startups and so just like giving you all the context there. Um, what tools that I have, like LinkedIn recruiter, of course, like am on that every single day. Um, in addition to that, the other tool that we use for sourcing is called Findem, which is a more like diversity focused, um, tool, which is really, really cool. Um, and it's nice to see the tool evolving as we've been using it. Um, in addition to that, definitely have like some agencies that I've worked with in the